In [30]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 例外処理用のライブラリをインポート
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
import sys

import pandas as pd
import numpy as np

seleniumマネジャーが利用可能かハージョン確認  
4.6以前であれば更新

In [31]:
print(selenium.__version__)

4.15.2


## 都道府県ごとのランキングを取得する関数

In [32]:
# 都道府県ごとのランキングデータフレームを作成する
def Pref_cycle(df_pref):
  for i in range(1, 48):
    # 都道府県ごとのデータ取得
    pref_element = driver.find_element(By.XPATH, '//div[@class="wrap"]/nav/div[@class="left"]/select[@name="state"]')
    # 都道府県のselect要素をselectオブジェクトへ変換
    pref_select = Select(pref_element)
    # オブジェクトからvalue属性のリストを作成する
    prefvalue_list = [pref.get_attribute('value') for pref in pref_select.options]
    # 都道府県の要素をドロップダウンから選択して
    pref_select.select_by_value(prefvalue_list[i])
    # 表示ボタンの要素を取得できるまで最大3秒間待機し、2秒ごとに要素が表示されるか確認
    viewer = WebDriverWait(driver, 3, poll_frequency=2).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
    )
    # 要素（表示ボタン）が表示されるまでスクロールする
    driver.execute_script("arguments[0].scrollIntoView();", viewer)
    viewer.click()
    # ページ全体がロードされるのを最大3秒間待機し、2秒ごとに確認
    WebDriverWait(driver, 3, poll_frequency=2).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )
    # top3の店名とポイント, top4~50の店名とポイントを取得できるまで最大3秒待機
    store_name_top3, store_point_top3, store_name, store_point = WebDriverWait(driver, 3).until(Check_namePoint_ele)
    # 取得したtop3の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_top3_list = [store_top3.text for store_top3 in store_name_top3]
    store_point_top3_list = [store_top3.text for store_top3 in store_point_top3]
    href_list_top3 = [ hrefs.get_attribute('href') for hrefs in store_name_top3]
    
    # 取得したtop4~50の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_list = [store.text for store in store_name]
    store_point_list = [store.text for store in store_point]
    href_list = [hrefs.get_attribute('href') for hrefs in store_name]
    Top3_detail(store_name_top3_list, store_point_top3_list, href_list_top3, df_pref, prefvalue_list[i])
    Top4to50_detail(store_name_list, store_point_list, href_list, df_pref, prefvalue_list[i])

def Check_namePoint_ele(driver):
  # top3の店のみ店名とポイントを取得
  store_name_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a')
  store_point_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="point"]')
  # top4~50の店名とポイントを取得
  store_name = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a')
  store_point = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td[@class="point"]')
  return store_name_top3, store_point_top3, store_name, store_point

#絞込に従う、ランキングtop3店の詳細情報を取得する
def Top3_detail(name_list, points_list, href_list, df_pref, pref_name):
  # top3の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    # リストに保存したurlに遷移する
    Safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = Get_address()
    # 緯度経度の要素を取得する
    latitude,  longitude = Get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = Get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = Get_holiday()
    Mk_detail_list(name_list[i], points_list[i], address, latitude, longitude, parking, holiday, df_pref, pref_name)
        
        
#絞込に従う、ランキングtop4~5店の詳細情報を取得する
def Top4to50_detail(name_list, points_list, href_list, df_pref, pref_name):
  # top4~50の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    Safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = Get_address()
    # 緯度経度の要素を取得する
    latitude,  longitude = Get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = Get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = Get_holiday()
    Mk_detail_list(name_list[i], points_list[i], address, latitude, longitude, parking, holiday, df_pref, pref_name)
  # 定期的なキャッシュの削除
  driver.get('chrome://settings/clearBrowserData')
  # ランキングのページに戻り、都道府県のプルダウンを取得できるようにする
  driver.get('https://ramendb.supleks.jp/rank')
  # ページ全体がロードされるのを最大3秒間待機する
  WebDriverWait(driver, 3).until(
      lambda d: d.execute_script('return document.readyState') == 'complete'
  )
    
def Get_address():
  try:
    #addressの要素を2秒おきに確認しながら、最大3秒待機する
    address_ele = WebDriverWait(driver, 3, poll_frequency=2).until(
        EC.presence_of_element_located((By.XPATH, '//tbody/tr/td/span[@itemprop="address"]'))
    )
    address = address_ele.text
  # 住所が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    address = None
  return address
        
def Get_map():
  try:
    # mapの要素を確認しながら、最大3秒待機する。addressと同じページ内なので、確認秒数はデフォルトの500ミリ秒とする
    map_data_ele = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.ID, 'gpetitembedmap'))
    )
    map_data = map_data_ele.get_attribute('src')
    # srcの要素値を緯度経度だけ切り離す
    # q= という文字が入るので、スライスで取り除く
    map_data = map_data.split('&')[1][2:]
    # 緯度経度に分割する
    map_data = map_data.split(',')
    latitude = map_data[0]
    longitude = map_data[1]
  # 緯度経度が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    latitude = None
    longitude = None
  return latitude, longitude
        
def Get_parking():
  try:
    th_element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "駐車場")]'))
    )
    parking = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    parking = None
  return parking
        
def Get_holiday():
  try:
    th_element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "定休日")]'))
    )
    holiday = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    holiday = None
  return holiday

# 店舗名とポイント、店舗詳細を店舗ごとにリスト化する
def Mk_detail_list(name, points, address, latitude, longitude, parking, holiday, df_pref, pref_name):
  detail_list = [name]
  detail_list.append(points)
  detail_list.append(pref_name)
  detail_list.append(address)
  detail_list.append(latitude)
  detail_list.append(longitude)
  detail_list.append(parking)
  detail_list.append(holiday)
  print(detail_list)
  df_pref.loc[len(df_pref)] = detail_list

### 詳細ページに遷移するための関数
* ~~WebdriverExceptionが発生した場合に待機して、リロードを行う~~<br>
* ~~2間隔でページ全体が更新されたか、最大3秒間確認する~~
* 3秒以上の動画広告に対処するため、この関数ではページ遷移と、エラー発生時のリロードのみ行う。<br>
5回リロードしても同じエラーが発生する場合には強制終了を行う

In [33]:
def Safe_navigation(href, i=0):
    try:
        driver.get(href)
     # 例外オブジェクトを e としてキャッチ
    except Exception as e:
        if i >= 5:
            try:
                sys.exit()
            except SystemExit:
                print('ページ遷移不可のため、強制終了')
                raise
        else:
            print(f"Navigation error: {e}")
            i += 1
            Safe_navigation(href, i)

## 分類ごとのランキングを取得する関数

In [34]:
#top3の店名と4〜50の店名の要素をリスト形式で返却する
def Check_name_ele(driver):
    store_name_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a')
    store_name = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a')
    return store_name_top3, store_name

# 分類ごとのランキング表を作成する
def Kind_cycle(kinds_another_list, pager):
    for i in pager:
        # 分類別ごとのデータ取得
        # 分類別、スープ別のselect要素を取得する
        kinds_sel_ele = driver.find_element(By.XPATH, '//select[@name="style"]')
        # 分類別の要素をオブジェクトへ変換する
        kinds_sel_obj = Select(kinds_sel_ele)
        # オブジェクトの、value属性のリストを作成する
        kinds_val_list = [kind.get_attribute('value') for kind in kinds_sel_obj.options]
        kinds_tex_list = [kind.text for kind in kinds_sel_obj.options]
        # 都道府県の要素をドロップダウンから選択して
        kinds_sel_obj.select_by_value(kinds_val_list[i])
        # 表示ボタンの要素を取得できるまで待機（最大十秒）
        viewer = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
        )
        # 要素（表示ボタン）が表示されるまでスクロールする
        driver.execute_script("arguments[0].scrollIntoView();", viewer)
        viewer.click()
        
        # top3の店名と、top4~50の店名を取得できるまで2秒おきに確認しながら、最大3秒待機
        store_name_top3, store_name = WebDriverWait(driver, 3, poll_frequency=2).until(
            Check_name_ele
        )

        # 取得したtop3の店名をリスト形式で保存
        store_name_top3_list = [store_top3.text for store_top3 in store_name_top3]
        # 取得したtop4~50の店名をリスト形式で保存
        store_name_list = [store.text for store in store_name]
        
        kinds_another_list.append(store_name_top3_list + store_name_list)
    
    # 種別ごとのランキングをカラムにするため、numpy形式に変換して転置する
    kinds_another_nplist = np.array(kinds_another_list).T
    return kinds_another_nplist, kinds_tex_list


## 都道府県ごとのランキングを取得したい場合に実行する

In [35]:
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(5)
driver.get('https://ramendb.supleks.jp/rank')

# # データフレームの枠組みを作成する
columns_list_detail = ['name', 'point', 'pref', 'address', 'latitude', 'longitude', 'parking', 'holiday']
df_pref = pd.DataFrame(columns = columns_list_detail)
Pref_cycle(df_pref)
df_pref.to_csv('../data/pref_rank.csv',index=False)

['麺屋 彩未', '96.689', 'hokkaido', '〒062-0010 北海道札幌市豊平区美園十条5丁目3-12', '43.035250593806', '141.38190865517', '駐車場あり\n25台', '月曜、火曜水曜の夜、月2回不定休有']
['ラーメン二郎 札幌店', '95.231', 'hokkaido', '〒060-0806 北海道札幌市北区北六条西8-8-11', '43.0673949', '141.3438214', '駐車場なし', '日曜、祝日不定休']
['Japanese Ramen Noodle Lab Q', '94.329', 'hokkaido', '〒060-0001 北海道札幌市中央区北1条西2丁目1-3 りんどうビル B1F', '43.063436564372', '141.35422906812', '駐車場なし', '日曜日']
['すみれ 札幌中の島本店', '94.274', 'hokkaido', '〒062-0922 北海道札幌市豊平区中の島二条4-7-28', '43.032554808437', '141.3615985433', '駐車場あり\n15台', '不定休']
['中華そば カリフォルニア', '93.410', 'hokkaido', '〒003-0803 北海道札幌市白石区菊水3条4丁目4-6 ビバ菊水1F', '43.054359025053', '141.37686231164', '駐車場あり\n店横 1台', '金曜']
['いそのかづお', '92.967', 'hokkaido', '北海道札幌市中央区南5条西5丁目21 第2旭観光ビル1F', '43.0542253', '141.3507467', '駐車場なし', '日曜']
['men-eiji HIRAGISHI BASE', '92.496', 'hokkaido', '〒062-0932 北海道札幌市豊平区平岸2条11丁目1-12', '43.029399142819', '141.36641374726', '駐車場あり\n4台', '水曜']
['麺や 麗', '92.290', 'hokkaido', '〒066-0035 北海道千歳市高台4丁目2-10', '42.836895

## 種類ごとのランキングが取得したい場合に実行する

In [14]:
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(5)
driver.get('https://ramendb.supleks.jp/rank')

# 種別ごとのTOP50の店名のリストを格納するリストを作成
kinds_another_list = []
# 実行するプルダウンのリスト
pager = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16]

kinds_another_nplist, kinds_tex_list = Kind_cycle(kinds_another_list, pager)

kinds_tex_list.remove('全て')
while "その他" in kinds_tex_list:
    kinds_tex_list.remove("その他")

# 取得した種類別のランキングをカラム別にcsv形式に変換
df_kindsrank = pd.DataFrame(kinds_another_nplist, columns = kinds_tex_list)
df_kindsrank.to_csv("../data/kinds_of_rank.csv",index=False)